# Built-in Solvers in 3d

This uses the web-based 3d transient volume renderer by [Simon
Niedermayr](https://github.com/KeKsBoTer). 3d volume rendering is costly.
Likely, some of the demos below will only work reasonably fast on desktop GPUs.
If you encounter performance issues, try to spatially subsample the produced
trajectories, decrease the resolution of the IFrame, or increase parameter step
size within the volume rendering window.

In [59]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

In [60]:
import exponax as ex
from IPython.display import IFrame

The trajectories have to be written out as numpy arrays. This will be the
temporary folder they are placed in.

In [61]:
BASE_FOLDER = "_3d_trjs/"

In [62]:
import os

if not os.path.exists(BASE_FOLDER):
    os.makedirs(BASE_FOLDER)

## Linear PDEs

### Advection

In [63]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 48
DT = 0.01
# Can also supply a scalar to use the same value for both dimensions
VELOCITY = jnp.array([0.4, 1.0, -0.2])

advection_stepper = ex.stepper.Advection(
    3, DOMAIN_EXTENT, NUM_POINTS, DT, velocity=VELOCITY
)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)
u_0 = (
    jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
    * jnp.cos(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
    * jnp.sin(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT)
)

advection_trj = ex.rollout(advection_stepper, 40, include_init=True)(u_0)

jnp.savez(BASE_FOLDER + "advection_trj.npz", trj=advection_trj)

The below window will then ask you to load the trajectory file.

In [64]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Diffusion

In [65]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 48
DT = 0.01
# See next section for anisotropic diffusion
NU = 0.01

anisotropic_diffusion_stepper = ex.stepper.Diffusion(
    3, DOMAIN_EXTENT, NUM_POINTS, DT, diffusivity=NU
)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)
u_0 = (
    jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
    * jnp.cos(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
    * jnp.sin(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT)
)

diffusion_trj = ex.rollout(anisotropic_diffusion_stepper, 40, include_init=True)(u_0)

jnp.savez(BASE_FOLDER + "diffusion_trj.npz", trj=diffusion_trj)

In [66]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Anisotropic Diffusion

In [67]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 48
DT = 0.01
# Can also supply a 2d vector for diagonal diffusivity. For full anisotropy, the
# matrix must be positive definite.
NU = jnp.array([[0.005, 0.003, 0.002], [0.003, 0.04, 0.001], [0.002, 0.001, 0.02]])

anisotropic_diffusion_stepper = ex.stepper.Diffusion(
    3, DOMAIN_EXTENT, NUM_POINTS, DT, diffusivity=NU
)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)
u_0 = (
    jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
    * jnp.cos(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
    * jnp.sin(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT)
)

anisotropic_diffusion_trj = ex.rollout(
    anisotropic_diffusion_stepper, 40, include_init=True
)(u_0)

jnp.savez(BASE_FOLDER + "anisotropic_diffusion_trj.npz", trj=anisotropic_diffusion_trj)

In [68]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Advection-Diffusion

In [69]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 48
DT = 0.01
# Can supply up to a vector for the advection speed, and up to a matrix for
# anisotropic diffusion
velocity = 1.0
diffusivity = 0.03

advection_diffusion_stepper = ex.stepper.AdvectionDiffusion(
    3, DOMAIN_EXTENT, NUM_POINTS, DT, velocity=velocity, diffusivity=diffusivity
)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)
u_0 = (
    jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
    * jnp.cos(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
    * jnp.sin(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT)
)

advection_diffusion_trj = ex.rollout(
    advection_diffusion_stepper, 40, include_init=True
)(u_0)

jnp.savez(BASE_FOLDER + "advection_diffusion_trj.npz", trj=advection_diffusion_trj)

In [70]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Dispersion

In [71]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 48
DT = 0.01
# Can also supply a vector for different dispersivity per dimension
DISPERSIVITY = 0.01

dispersion_stepper = ex.stepper.Dispersion(
    3, DOMAIN_EXTENT, NUM_POINTS, DT, dispersivity=DISPERSIVITY
)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)
u_0 = jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT) * jnp.cos(
    2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT
) + jnp.sin(3 * 2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT) * jnp.cos(
    3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT
)
dispersion_trj = ex.rollout(dispersion_stepper, 40, include_init=True)(u_0)

jnp.savez(BASE_FOLDER + "dispersion_trj.npz", trj=dispersion_trj)

In [72]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Hyper-Diffusion

In [73]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 48
DT = 0.01
HYPER_DIFFUSIVITY = 0.0001

hyper_diffusion_stepper = ex.stepper.HyperDiffusion(
    3, DOMAIN_EXTENT, NUM_POINTS, DT, hyper_diffusivity=HYPER_DIFFUSIVITY
)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)
u_0 = (
    jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
    * jnp.cos(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
    * jnp.sin(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT)
)

hyper_diffusion_trj = ex.rollout(hyper_diffusion_stepper, 40, include_init=True)(u_0)

jnp.savez(BASE_FOLDER + "hyper_diffusion_trj.npz", trj=hyper_diffusion_trj)

In [74]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

## Nonlinear PDEs

In [75]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 48
DT = 0.01
NU = 0.01

burgers_stepper = ex.stepper.Burgers(3, DOMAIN_EXTENT, NUM_POINTS, DT, diffusivity=NU)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)

# Burgers has two channels!
u_0 = jnp.concatenate(
    [
        jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
        * jnp.cos(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
        * jnp.sin(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT),
        jnp.cos(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
        * jnp.sin(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
        * jnp.cos(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT),
        jnp.cos(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT),
    ]
)

burgers_trj = ex.rollout(burgers_stepper, 40, include_init=True)(u_0)

jnp.savez(BASE_FOLDER + "burgers_trj.npz", trj=burgers_trj)

In [76]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Single-Channel Burgers

In [77]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 48
DT = 0.01
NU = 0.01

single_channel_burgers_stepper = ex.stepper.Burgers(
    3, DOMAIN_EXTENT, NUM_POINTS, DT, diffusivity=NU, single_channel=True
)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)

u_0 = (
    jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
    * jnp.cos(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
    * jnp.sin(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT)
)

single_channel_burgers_trj = ex.rollout(
    single_channel_burgers_stepper, 40, include_init=True
)(u_0)

jnp.savez(
    BASE_FOLDER + "single_channel_burgers_trj.npz", trj=single_channel_burgers_trj
)

In [78]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Kuramoto-Sivashinsky (KS)

In [79]:
DOMAIN_EXTENT = 30.0
NUM_POINTS = 48
DT = 0.5
NUM_SUBSTEPS = 5

ks_stepper = ex.RepeatedStepper(
    ex.stepper.KuramotoSivashinsky(3, DOMAIN_EXTENT, NUM_POINTS, DT / NUM_SUBSTEPS),
    NUM_SUBSTEPS,
)

# IC is irrelevant
u_0 = jax.random.normal(jax.random.PRNGKey(0), (1, NUM_POINTS, NUM_POINTS, NUM_POINTS))

u_0 -= jnp.mean(u_0)

warmed_up_u_0 = ex.repeat(ks_stepper, 500)(u_0)

ks_trj = ex.rollout(ks_stepper, 40, include_init=True)(warmed_up_u_0)

jnp.savez(BASE_FOLDER + "ks_trj.npz", trj=ks_trj)

In [80]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Korteweg-de Vries (KdV)

In [81]:
DOMAIN_EXTENT = 20.0
NUM_POINTS = 48
DT = 0.05

kdv_stepper = ex.stepper.KortewegDeVries(
    3,
    DOMAIN_EXTENT,
    NUM_POINTS,
    DT,
    single_channel=True,
)

grid = ex.make_grid(3, DOMAIN_EXTENT, NUM_POINTS)
u_0 = (
    jnp.sin(2 * jnp.pi * grid[0:1] / DOMAIN_EXTENT)
    * jnp.cos(2 * 2 * jnp.pi * grid[1:2] / DOMAIN_EXTENT)
    * jnp.sin(3 * 2 * jnp.pi * grid[2:3] / DOMAIN_EXTENT)
)

kdv_trj = ex.rollout(kdv_stepper, 40, include_init=True)(u_0)

jnp.savez(BASE_FOLDER + "kdv_trj.npz", trj=kdv_trj)

In [82]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

## Reaction-Diffusion Systems

In [83]:
DOMAIN_EXTENT = 10.0
NUM_POINTS = 96
DT = 0.01
DIFFUSIVITY = 0.01
REACTIVITY = 10.0

fisher_kpp_stepper = ex.reaction.FisherKPP(
    3, DOMAIN_EXTENT, NUM_POINTS, DT, diffusivity=DIFFUSIVITY, reactivity=REACTIVITY
)

ic_gen = ex.ic.ClampingICGenerator(ex.ic.RandomTruncatedFourierSeries(3), limits=(0, 1))
u_0 = ic_gen(NUM_POINTS, key=jax.random.PRNGKey(0))

fisher_kpp_trj = ex.rollout(fisher_kpp_stepper, 40, include_init=True)(u_0)

# Spatial substepping to get 48^3
fisher_kpp_trj = fisher_kpp_trj[..., ::2, ::2, ::2]

jnp.savez(BASE_FOLDER + "fisher_kpp_trj.npz", trj=fisher_kpp_trj)

In [84]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Gray-Scott

In [85]:
DOMAIN_EXTENT = 1.0
NUM_POINTS = 96
DT = 50.0
DIFFUSIVITY_0 = 2e-5
DIFFUSIVITY_1 = 1e-5
FEED_RATE = 0.04
KILL_RATE = 0.06

gray_scott_stepper = ex.RepeatedStepper(
    ex.reaction.GrayScott(
        3,
        DOMAIN_EXTENT,
        NUM_POINTS,
        DT / 50,
        diffusivity_1=DIFFUSIVITY_0,
        diffusivity_2=DIFFUSIVITY_1,
        feed_rate=FEED_RATE,
        kill_rate=KILL_RATE,
    ),
    50,
)

u_0 = ex.ic.RandomMultiChannelICGenerator(
    [
        ex.ic.RandomGaussianBlobs(3, one_complement=True),
        ex.ic.RandomGaussianBlobs(3),
    ]
)(NUM_POINTS, key=jax.random.PRNGKey(0))

gray_scott_trj = ex.rollout(gray_scott_stepper, 40, include_init=True)(u_0)

# Spatial substepping to get 48^3
gray_scott_trj = gray_scott_trj[..., ::2, ::2, ::2]

jnp.savez(BASE_FOLDER + "gray_scott_trj.npz", trj=gray_scott_trj)

In [86]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

### Swift-Hohenberg

In [87]:
DOMAIN_EXTENT = 20.0 * jnp.pi
NUM_POINTS = 96
DT = 1.0

swift_hohenberg_stepper = ex.RepeatedStepper(
    ex.reaction.SwiftHohenberg(3, DOMAIN_EXTENT, NUM_POINTS, DT / 10),
    10,
)

u_0 = ex.ic.RandomTruncatedFourierSeries(3, max_one=True)(
    NUM_POINTS, key=jax.random.PRNGKey(0)
)

swift_hohenberg_trj = ex.rollout(swift_hohenberg_stepper, 40, include_init=True)(u_0)

# Spatial substepping to get 48^3
swift_hohenberg_trj = swift_hohenberg_trj[..., ::2, ::2, ::2]

jnp.savez(BASE_FOLDER + "swift_hohenberg_trj.npz", trj=swift_hohenberg_trj)

In [88]:
IFrame(src="https://keksboter.github.io/v4dv", width=1200, height=800)

## All together

In [89]:
joint_trj = jnp.concatenate(
    [
        advection_trj,
        diffusion_trj,
        anisotropic_diffusion_trj,
        advection_diffusion_trj,
        dispersion_trj,
        hyper_diffusion_trj,
        burgers_trj,
        single_channel_burgers_trj,
        ks_trj,
        kdv_trj,
        fisher_kpp_trj,
        gray_scott_trj,
        swift_hohenberg_trj,
    ],
    axis=1,
)

jnp.savez(BASE_FOLDER + "joint_trj.npz", trj=joint_trj)

In [90]:
IFrame(src="https://keksboter.github.io/v4dv", width=1800, height=1000)